# Data Collection

Load in libraries and datasets. The datasets come from Kaggle, internet sources, and a basketball reference scrape.

In [3]:
# read in libraries
import pandas as pd
import numpy as np
import pickle
pd.options.mode.chained_assignment = None 
pd.errors.DtypeWarning = None 

In [4]:
# read in datasets

# kaggle datasets
games = pd.read_csv('data/games.csv') 
games_details = pd.read_csv('data/games_details.csv')
players = pd.read_csv('data/players.csv')
ranking = pd.read_csv('data/ranking.csv')
teams = pd.read_csv('data/teams.csv')

# datasets created from internet
jersey = pd.read_csv('data/jersey2.csv')
twok = pd.read_csv('data/2kRank.csv')
pacedata = {'season':[2012,2013,2014,2015,2016,2017,2018,2019,2020],
           'pace':[92.0,93.9,93.9,95.8,96.4,97.3,100.0,100.3,99.2]}
pace = pd.DataFrame(pacedata)

# load in scraped all star info
all_star_appearances = pickle.load(open('data/all_star_appearances.pickle', 'rb'))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Data Preparation

Merge date to the individual games dataset (one row for every player in every game) and use month and date to label each row with a season. The convention used is the first year in the season is the name of the season (i.e. 2015-2016 is the 2015 season). Then, filter for the relevant time periods, which are the months September through January for the 2012 - 2020 seasons. The 2020 NBA Championship games must then be removed because they took place in September and Ocotber due to postponements caused by COVID-19. There is also an issue with the 2020 data being duplicated so duplicates are removed.

In [5]:
# join date to individual games stats dataset
games_details2 = pd.merge(games_details, games[['GAME_DATE_EST', 'GAME_ID']], on = 'GAME_ID', how = 'left')

# extract year and month for every row
games_details2['year'] =  pd.to_numeric(games_details2['GAME_DATE_EST'].apply(lambda x: x.split('-')[0]))
games_details2['month'] =  pd.to_numeric(games_details2['GAME_DATE_EST'].apply(lambda x: x.split('-')[1]))

# label each row with season based on year and month
games_details2['season'] = games_details2[['year','month']].apply(lambda x: x['year']-1 if (x['month']==1) else x['year'], axis=1)

# keep only game info for September through January in the 2012-2020 seasons
games_details2 = games_details2[games_details2['month'].isin([9,10,11,12,1])]
games_details3 = games_details2[(games_details2['season']<2021) & (games_details2['season']>2011)]

# get rid of 2020 finals
games_details3 = games_details3[~((games_details3['season']==2020) & (games_details3['month'].isin([9,10])))]

# drop duplicates
games_details3 = games_details3.drop_duplicates(subset=['GAME_ID','PLAYER_ID'])

Prepares the individual games dataset to be summarized by player and season. It does this by getting rid of rows where the players did not play, getting rid of one erroneous row, extracting the number of minutes they played, and adding a helper column to indicate the player did play in the game.

In [6]:
# get rid of rows with comments (means they did not play)
games_details3 = games_details3[games_details3['COMMENT'].isnull()]

# get rid of one erroneous row
games_details3 = games_details3[games_details3['PTS'].notnull()]

# extract number of minutes played
games_details3['minutes'] =  pd.to_numeric(games_details3['MIN'].apply(lambda x: x.split(':')[0]))

# add a column to indicate they played in that game (useful for aggregation in next step)
games_details3['GP'] = 1

Finds the summary statistics by player and season for all the basic stats collected using groupby. This dataset forms the basis for the main dataset going forward.

In [7]:
# uses groupby to summarize by player and season
grouped = games_details3.groupby(['PLAYER_ID','season']).apply(lambda s: pd.Series({ 
    "PLAYER_NAME": s["PLAYER_NAME"].mode()[0],
    "TEAM_ABBREVIATION": s["TEAM_ABBREVIATION"].mode()[0],
    "TEAM_ID": s["TEAM_ID"].mode()[0],
    "GP": s["GP"].sum(),
    "Minutes": s["minutes"].mean(),
    "FGM": s["FGM"].mean(),
    "FGA": s["FGA"].mean(),
    "FG3M": s["FG3M"].mean(),
    "FG3A": s["FG3A"].mean(),
    "FTM": s["FTM"].mean(),
    "FTA": s["FTA"].mean(),
    "OREB": s["OREB"].mean(),
    "DREB": s["DREB"].mean(),
    "AST": s["AST"].mean(),
    "STL": s["STL"].mean(),
    "BLK": s["BLK"].mean(),
    "TO": s["TO"].mean(), 
    "PF": s["PF"].mean(),
    "PTS": s["PTS"].mean(),
    "PLUS_MINUS": s["PLUS_MINUS"].mean(),
    "Starting_Position": s["START_POSITION"].mode(),
})).reset_index()

Fix Starting_Position column so that the most common starting position for each player is displayed correctly and those with no information are labeled with "None".

In [8]:
# Fix Starting_Position column
grouped['Starting_Position'] = grouped['Starting_Position'].apply(lambda x: "None" if len(x)==0 else x[0])

Adjust each season for pace by merging the pace data and dividing the relevant columns by the pace/100.

In [9]:
# merge pace data
grouped = pd.merge(grouped, pace, on = ['season'], how = 'left')

# divide pace data by 100
grouped['pace'] = grouped['pace']/100

# divide relevant statistics by pace/100
grouped[['FGM','FGA','FG3M','FG3A','FTM','FTA','OREB',
       'DREB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS']] = grouped[['FGM','FGA','FG3M','FG3A','FTM','FTA','OREB',
       'DREB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS']].div(grouped.pace, axis=0)

Calculate Advanced Metrics PER and True Shooting Percentage.

In [10]:
# PER
grouped['PER'] = (grouped['FGM']*85.910 + grouped['STL']*53.897 + 
grouped['FG3M']*51.757 + grouped['FTM']*46.845 + grouped['BLK']*39.190
+ grouped['OREB']*39.190+ grouped['AST']*34.677 + grouped['DREB']*14.707
- grouped['PF']*17.174 - (grouped['FTA']-grouped['FTM'])*20.091 - (grouped['FGA']-grouped['FGM'])*39.190
- grouped['TO']*53.897)*(1/grouped['Minutes'])

# True Shooting Percentage
grouped['TSP'] = grouped['PTS']/(2*(grouped['FGA']+.44*grouped['FTA']))

Add each player's team win percentage and team conference for each season as of January 20. This is done first by keeping only the team ranking data as of January 20 for each season and adjusting the season ID to match the season. Then, this data is merged with the main data.

In [11]:
# keep only January 20 data
ranking2 = ranking[(ranking['STANDINGSDATE'].apply(lambda x: x.split('-')[1])=='01') & (ranking['STANDINGSDATE'].apply(lambda x: x.split('-')[2])=='20')]

# get season from the year
ranking2['season'] = ranking2['SEASON_ID']-20000
grouped = pd.merge(grouped, ranking2[['TEAM_ID', 'season','CONFERENCE','W_PCT']], on = ['TEAM_ID','season'], how = 'left')

Add if a player's jersey was in the top 15 of best sellers the previous year by merging with jersey data.

In [12]:
# merge jersey data
grouped = pd.merge(grouped, jersey, on = ['season','PLAYER_ID'], how = 'left')

Add the players' 2KRank for that season (released before the year starts).

In [13]:
# merge 2k data
grouped = pd.merge(grouped, twok[['PLAYER_ID','season','2KRank']],on=['PLAYER_ID','season'],how='left')

Adds All Star information for each player/season. Specifically, it adds wether a player was an All Star the previous year, the total amount of prior All Star selections a player had, and if a player was selected as an All Star that season (this is the target).

In [14]:
# function that defines if a player was an All star the previous year
def was_AS_last_year(row):
    if row['season'] == 1999:
        return 1 if 1998 in all_star_appearances[row['PLAYER_NAME']] else 0
    return 1 if row['season'] in all_star_appearances[row['PLAYER_NAME']] else 0

# adds if a player was an ALl star the previous year
grouped['LastASG?'] = grouped[['PLAYER_NAME', 'season']].apply(was_AS_last_year, axis=1)

# adds the total amount of prior selections a player had as of that year (not including that year)
grouped['PriorASG'] = grouped[['PLAYER_NAME', 'season']].apply(lambda row : sum(y<=row['season'] for y in all_star_appearances[row['PLAYER_NAME']]), axis=1)

# adds wether a player was selected as an all star that year
grouped['Selected?'] = grouped[['PLAYER_NAME', 'season']].apply(lambda row : 1 if row['season']+1 in all_star_appearances[row['PLAYER_NAME']] else 0, axis=1)

Adjusts for variability in games played up to that point in the season (especially relevant in 2020 season) by calculating the percentage of his team's games a player participated in.

In [15]:
# get total games played by each team up to that point in each season
game_count1 = games_details3.groupby(['season','GAME_ID','TEAM_ID']).GP.mean().reset_index()
game_count2 = game_count1.groupby(['season','TEAM_ID']).GP.sum().reset_index()

# merges Games Played rate with main dataset
grouped = pd.merge(grouped,game_count2,on=['season','TEAM_ID'],how='left')

# calculates percentage games played
grouped['GP_Per'] = grouped['GP_x'] / grouped['GP_y']

Null data is dealt with next, setting missing stats values to zero and giving unranked players a very low rank.

In [16]:
#fill null values
grouped['PLUS_MINUS'] = grouped['PLUS_MINUS'].fillna(0)
grouped['TSP'] = grouped['TSP'].fillna(0)
grouped['PER'] = grouped['PER'].fillna(0)
grouped['2KRank'] = grouped['2KRank'].fillna(500)

In [17]:
grouped.isnull().sum(axis = 0)

PLAYER_ID            0
season               0
PLAYER_NAME          0
TEAM_ABBREVIATION    0
TEAM_ID              0
GP_x                 0
Minutes              0
FGM                  0
FGA                  0
FG3M                 0
FG3A                 0
FTM                  0
FTA                  0
OREB                 0
DREB                 0
AST                  0
STL                  0
BLK                  0
TO                   0
PF                   0
PTS                  0
PLUS_MINUS           0
Starting_Position    0
pace                 0
PER                  0
TSP                  0
CONFERENCE           0
W_PCT                0
Jersey               0
2KRank               0
LastASG?             0
PriorASG             0
Selected?            0
GP_y                 0
GP_Per               0
dtype: int64

In the 2018 season All Star game, Dirk Nowitzki and Dwyane Wade were added as special additions to honor their legacy. They are removed from the dataset to not skew the data.

In [18]:
grouped.loc[(grouped['PLAYER_NAME']=='Dwyane Wade') & (grouped['season']==2018),'Selected?'] = 0
grouped.loc[(grouped['PLAYER_NAME']=='Dirk Nowitzki') & (grouped['season']==2018),'Selected?'] = 0

### EDA

All Star Participants are picked on a variety of factors, including popular vote, media representation, and peer opinions. A large assumption we made in our project was that these popular or public opinions correlated highly with player statistics. We chose to only consider player statistics in our analysis, because these statistics tend to be highly regarded by the community, and are a good representation of how well a player does, in certain respects. 

We also used popularity data like Jersey sales to further inform our analysis.

Our data set includes the following columns:

In [20]:
grouped.columns

Index(['PLAYER_ID', 'season', 'PLAYER_NAME', 'TEAM_ABBREVIATION', 'TEAM_ID',
       'GP_x', 'Minutes', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB',
       'DREB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS',
       'Starting_Position', 'pace', 'PER', 'TSP', 'CONFERENCE', 'W_PCT',
       'Jersey', '2KRank', 'LastASG?', 'PriorASG', 'Selected?', 'GP_y',
       'GP_Per'],
      dtype='object')

In [29]:
# random eda

#grouped[(grouped['Selected?'] == 1) & (grouped['season'] == 2018)]
#grouped[grouped['PLAYER_NAME']=='Klay Thompson']
#grouped[grouped['PTS']>20].sort_values(by=['W_PCT'],ascending=False).head(10)
grouped['LastASG?']

0       0
1       0
2       0
3       0
4       1
       ..
5011    0
5012    0
5013    0
5014    0
5015    0
Name: LastASG?, Length: 5016, dtype: int64

The stats are computed as average per game, unless otherwise noted.
Here is a table with the column definitions

| Column/Variable Name | Definition |
|----------------------|----------- |
|`PLAYER_ID`| Unique identifier for the player |
|`season`| Season or year of the data |
|`PLAYER_NAME` | Name of the player |
|`TEAM_ABBREVIATION`| The team the player is playing for|
|`TEAM_ID`| Unique identifier for the player|
|`GP_x`| *blank* |
|`Minutes` | Number of minutes the player was in the game|
|`FGM` | Field Goals made (2-pointers and 3-pointers)|
|`FGA` | Field Goals attempted (2-pointers and 3-pointers)|
|`FG3M` | 3-pointers made |
|`FG3A` | 3-pointers atempted |
|`FTM` | Free Throws made |
|`FTA`| Free Throws attempted |
|`OREB`| The number of rebounds a player or team has collected while they were on offense |
|` DREB`| The number of rebounds a player or team has collected while they were on defense |
|`AST`| The number of assists made |
|`STL`| The number of steals made while on defense |
|`BLK` | The number of successful blocks made while on defense |
|`TO` | The number of turnovers |
|`PF` | Number of Personal fouls |
|`PTS` | Number of Points the player scored |
|`PLUS_MINUS`| *blank* |
|`Starting_Position`| The position the player most often started in|
|`pace` | The number of possessions per 48 minutes for a player|
|`PER` | An advanced metric used to rank players |
|`TSP` | True Shooting percentage |
|`CONFERENCE` | Which conference the player is in |
|`W_PCT` | *blank*|
|`Jersey` | Boolean to indicate whether the players jersey was in the top 15 in sales |
|`2KRank` | The overall 2K rank of the player before the season started |
|`LastASG?` | Boolean of whether the player participated in the season prior's All Star game |
|`PriorASG?` | Boolean of whether the player has participated in an All Star game prior to the current season |
|`Selected?`| Whether the player was selected for the current year's All Star Game |
|`GP_y` | *blank* |
|`GP_Per` | Percentage of games played in the season |


Let's take a look at some of the stats, and how they are spread: